In [1]:
import os
os.chdir("/home/data/workspace/heqi/matchingloss")
from depthnet.utils import *
from depthnet.model import EstimateDepth
from pathlib import Path

# path of model
model_name = "simcol_complete/RCC_cropalign_matching_depthnet_simcol_complete_monodepth2_rep6"

device = f'cuda:0'
gpu_id = 0
if gpu_id is not None:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
if torch.cuda.is_available():
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


cfgs = load_yaml(Path("results") / model_name / "models" / "configs.yml")
cfgs.update({'device': device, "not_load_nets": ["net_pose_encoder", "net_pose_decoder"]})


def getDepthNet():
    cp_path = Path('results') / model_name / "models" / "weights_last"

    model = EstimateDepth(cfgs)
    
    cp = {}
    for network_name in model.network_names:
        cp[network_name] = torch.load(cp_path / "{}.pth".format(network_name), map_location=device)

    model.load_model_state(cp)

    return model

model = getDepthNet()
model.to_device(device)

# load
import os
import torch
from torch.utils.data.dataloader import default_collate
import collections

import cv2
import numpy as np
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

from PIL import Image
import cv2

import glob

from depthnet.networks.layers import disp_to_depth
from depthnet.utils import readlines
import depthnet.datasets as datasets
import depthnet.networks as networks
from depthnet.datasets import C3VDDataset, SimcolDataset
    
exps = [cfgs["model_name"]]

modes = ['test_seperately/test_S_I_S5_files.txt',
        'test_seperately/test_S_I_S15_files.txt',
        'test_seperately/test_S_I_S10_files.txt',
        'test_seperately/test_S_II_B5_files.txt',
        'test_seperately/test_S_II_B15_files.txt',
        'test_seperately/test_S_II_B10_files.txt',
        'test_seperately/test_S_III_O1_files.txt',
        'test_seperately/test_S_III_O2_files.txt',
        'test_seperately/test_S_III_O3_files.txt']

test_align_paper = []

test_mean_errors = []
meds = []
stds = []
masses = []

warning1 = False
warning2 = False


for mode in modes:

    gt_depths = []
    split = cfgs.get('split')
    fpath = os.path.join("splits", split, "{}")
    data_path = cfgs.get('data_path')
    filenames = readlines(fpath.format(mode))
    for line in tqdm(filenames):
        folder, _, frame_id, _ = line.split()
        frame_fpath = os.path.join(data_path, folder, "{}{}".format(frame_id.replace("FrameBuffer", "Depth"), ".png"))
        gt_depth = cv2.imread(frame_fpath, -1)/(255*256)
        gt_depths.append(gt_depth)
        
    gt_depths = np.array(gt_depths) * 20 # cm

    print(cfgs["model_name"])


    num_workers = cfgs.get('num_workers', 4)
    height = cfgs.get('height', 256)
    width = cfgs.get('width', 320)
    frame_ids = [0]
    num_scales = 1
    dataset = globals().get(cfgs.get('dataset', C3VDDataset))
    split = cfgs.get('split')
    fpath = os.path.join("splits", split, "{}")
    filenames = readlines(fpath.format(mode))
    img_ext = '.png' if cfgs.get('png', False) else '.jpg'
    matcher_result_load = None #np.load(cfgs.get('matcher_result', None), allow_pickle=True).all()

    dataset = dataset(data_path, filenames, matcher_result_load,
        height, width, frame_ids, num_scales,
        is_train=False, img_ext=img_ext)
    dataloader = DataLoader(dataset, 16, shuffle=False, num_workers=num_workers,
                            pin_memory=True, drop_last=False)

    model.set_eval()

    pred_depths = []

    print("-> Computing predictions with size {}x{}".format(width, height))

    with torch.no_grad():
        for data in tqdm(dataloader):
            input_color = data[("color", 0, 0)].to(device)

            output = model.net_depth_decoder(model.net_depth_encoder(input_color))

            _, pred_depth = disp_to_depth(output[("disp", 0)], cfgs["min_depth"], cfgs["max_depth"])
            pred_depth = pred_depth.cpu()[:, 0].numpy()
            pred_depths.append(pred_depth)
            
    pred_depths = np.concatenate(pred_depths)

    MIN_DEPTH = 0.001
    MAX_DEPTH = 20.
    errors = []
    ratios = []

    def compute_errors(gt, pred):
        thresh = np.maximum((gt / pred), (pred / gt))
        a1 = (thresh < 1.25     ).mean()
        a2 = (thresh < 1.25 ** 2).mean()
        a3 = (thresh < 1.25 ** 3).mean()

        rmse = (gt - pred) ** 2
        rmse = np.sqrt(rmse.mean())

        rmse_log = (np.log(gt) - np.log(pred)) ** 2
        rmse_log = np.sqrt(rmse_log.mean())

        abs_rel = np.mean(np.abs(gt - pred) / gt)

        sq_rel = np.mean(((gt - pred) ** 2) / gt)

        return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3
    
    def eval_depth(pred, gt_depth):
        L1_error = np.mean(np.abs(pred - gt_depth))
        rel_error = np.median(np.abs((pred - gt_depth)/(gt_depth + 10e-5))) * 100
        RMSE_error = np.sqrt(np.mean((pred - gt_depth)**2))
        return L1_error, rel_error, RMSE_error
    
    gts_ = np.mean(np.mean(gt_depths, 1), 1)
    preds_ = np.mean(np.mean(np.array(pred_depths), 1), 1)
    scale = np.sum(preds_ * gts_) / np.sum(preds_ * preds_)  # monocular methods predict depth up to scale
    print('Scale: ', scale)
    L1_errors, rel_errors, rmses = [], [], []
    
    for i in range(pred_depths.shape[0]):
        gt_depth = gt_depths[i]
        gt_height, gt_width = gt_depth.shape[:2]
        pred_depth = cv2.resize(pred_depths[i], (gt_width, gt_height)) * scale
        
        pred_depth[pred_depth < MIN_DEPTH] = MIN_DEPTH
        pred_depth[pred_depth > MAX_DEPTH] = MAX_DEPTH
        
        L1_error, rel_error, rmse = eval_depth(pred_depth, gt_depth)
        L1_errors.append(L1_error)
        rel_errors.append(rel_error)
        rmses.append(rmse)
        
    print('Mean L1 error in cm: ', np.mean(L1_errors))
    print('Median relative error in %: ', np.mean(rel_errors))
    print('Mean RMSE in cm: ', np.mean(rmses))
    
    test_align_paper.append([np.mean(L1_errors), np.mean(rel_errors), np.mean(rmses)])

    for i in range(pred_depths.shape[0]):
        gt_depth = gt_depths[i]
        gt_height, gt_width = gt_depth.shape[:2]

        pred_depth = pred_depths[i]
        pred_depth = cv2.resize(pred_depth, (gt_width, gt_height))

        mask = gt_depth > 0

        pred_depth = pred_depth[mask]
        gt_depth = gt_depth[mask]

        ratio = np.median(gt_depth) / np.median(pred_depth)
        ratios.append(ratio)
        pred_depth *= ratio

        pred_depth[pred_depth < MIN_DEPTH] = MIN_DEPTH
        pred_depth[pred_depth > MAX_DEPTH] = MAX_DEPTH

        errors.append(compute_errors(gt_depth, pred_depth))

    print("gt_width, gt_height", gt_width, gt_height)

    ratios = np.array(ratios)
    med = np.median(ratios)
    print(" Scaling ratios | med: {:0.3f} | std: {:0.3f}".format(med, np.std(ratios / med)))

    mean_errors = np.array(errors).mean(0)

    print("\n  " + ("{:>8} | " * 7).format("abs_rel", "sq_rel", "rmse", "rmse_log", "a1", "a2", "a3"))
    print(("&{: 8.3}  " * 7).format(*mean_errors.tolist()) + "\\\\")
    print("\n-> Done!")
    
    test_mean_errors.append(mean_errors)
    meds.append(med)
    stds.append(np.std(ratios / med))
    masses.append(len(dataset))

print(cfgs["model_name"])
print("meds: " + ("{:0.3f}  " * len(meds)).format(*meds))
mean_med = np.sum([mass * med for mass, med in zip(masses, meds)]) / np.sum(masses)
# print("mean med: " + ("{:0.3f}").format(mean_med))
mean_std = np.sum([mass * std for mass, std in zip(masses, stds)]) / np.sum(masses)
# print("mean std: {}".format(mean_std))
total_mean_errors = np.sum([mass * mean_errors for mass, mean_errors in zip(masses, test_mean_errors)], axis=0) / np.sum(masses)


print("\n  " + ("{:>8} | " * 9).format("abs_rel", "sq_rel", "rmse", "rmse_log", "a1", "a2", "a3", "med", "std"))
print(("&{: 8.3}  " * 7).format(*total_mean_errors.tolist()) + "&{: 8.3}  &{: 8.3}  ".format(mean_med, mean_std) + "\\\\")
print("\n-> Done!")




/home/data/workspace/heqi/mmcv/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


Loading configs from results/simcol_complete/RCC_cropalign_matching_depthnet_simcol_complete_monodepth2_rep6/models/configs.yml
Loading  net_depth_encoder
Loading  net_depth_decoder


  0%|          | 0/1195 [00:00<?, ?it/s]

simcol_complete/RCC_cropalign_matching_depthnet_simcol_complete_monodepth2_rep6
-> Computing predictions with size 352x352


  0%|          | 0/75 [00:00<?, ?it/s]

...

Scale:  0.9666702856276023
Mean L1 error in cm:  0.1759175063839009
Median relative error in %:  4.04584563221887
Mean RMSE in cm:  0.34646505766587177
gt_width, gt_height 475 475
 Scaling ratios | med: 0.954 | std: 0.025

   abs_rel |   sq_rel |     rmse | rmse_log |       a1 |       a2 |       a3 | 
&   0.053  &  0.0272  &   0.362  &  0.0899  &   0.976  &   0.993  &   0.997  \\

-> Done!


  0%|          | 0/595 [00:00<?, ?it/s]

simcol_complete/RCC_cropalign_matching_depthnet_simcol_complete_monodepth2_rep6
-> Computing predictions with size 352x352


  0%|          | 0/38 [00:00<?, ?it/s]

Scale:  0.966243955850548
Mean L1 error in cm:  0.18336595379900353
Median relative error in %:  4.3874391948073725
Mean RMSE in cm:  0.352922362602129
gt_width, gt_height 475 475
 Scaling ratios | med: 0.947 | std: 0.027

   abs_rel |   sq_rel |     rmse | rmse_log |       a1 |       a2 |       a3 | 
&  0.0558  &  0.0282  &    0.37  &   0.091  &   0.974  &   0.994  &   0.997  \\

-> Done!
simcol_complete/RCC_cropalign_matching_depthnet_simcol_complete_monodepth2_rep6
meds: 0.883  0.887  0.891  0.965  0.962  0.961  0.956  0.954  0.947  

   abs_rel |   sq_rel |     rmse | rmse_log |       a1 |       a2 |       a3 |      med |      std | 
&  0.0754  &  0.0658  &   0.374  &   0.106  &   0.958  &   0.987  &   0.994  &    0.93  &  0.0468  \\

-> Done!
